In [1]:
import keras.datasets.imdb

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

In [69]:
def decode_sentence(review):
    word_index = keras.datasets.imdb.get_word_index()
    inverted_word_index = dict((i, word) for (word, i) in word_index.items())
    decoded_sequence = " ".join(inverted_word_index[i] for i in review)
    return decoded_sequence

decode_sentence(x_train[0])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

In [30]:
print(type(word_index.values()))
print(type(list(word_index.values())))

<class 'dict_values'>
<class 'list'>


In [6]:
import numpy as np

# utilisation de numpy pour verifier que la separation des jeux a ete correctement effectue

print(type(x_train))
np.array(x_train).shape

<class 'numpy.ndarray'>


(25000,)

In [7]:
def get_total_length(x_train):
    res = 0
    for l in x_train:
        res += len(l)
    return res

In [20]:
import math
from tqdm import tqdm

def naive_bayes_classifier_train(X, y):
    """
    Classificateur Bayesien naif
    Inputs: le trainset du dataset imdb
    Output: TODO
    """
    # tuple valeur/etiquette dans chaque element de imbd
    n_doc = len(X)
    nb_classes = len(np.unique(y))
    
    logprior = [0] * nb_classes
    bigdoc = [[]] * nb_classes
    vocabulary = list(keras.datasets.imdb.get_word_index().values())
    loglikelyhood = np.ndarray((len(vocabulary), nb_classes))
    
    for c in range(nb_classes):        
        # array numpy, donc masque
        c_review = X[y == c]
        nb_word_total  = get_total_length(c_review)
        n_c = len(c_review)
        logprior[c] = np.log(n_c / n_doc)
        
        #recuperation des cles sous formes numerisees
        voca_occurency = np.zeros(len(vocabulary))
        
        for docu in tqdm(c_review):
            voca_occurency = np.add(voca_occurency, np.histogram(docu, bins=np.arange(len(vocabulary) + 1))[0])
        
        for index in tqdm(range(len(vocabulary))):
            value = math.log((voca_occurency[index] + 1) /
                                      (nb_word_total + 1))
            loglikelyhood[index, c] = value
    return logprior, loglikelyhood, voca

In [21]:
(logprior, loglikelyhood, voca) = naive_bayes_classifier_train(x_train, y_train)

100%|██████████| 88584/88584 [00:00<00:00, 550702.89it/s]


In [64]:
def test_naive_bayes(testdoc, logprior, loglikelyhood, C, V):
    _sum = [0] * len(C)
    for c in range(len(C)):
        print(c)
        _sum[c] = logprior[c]
        for (i, word) in enumerate(testdoc):
            if word in V:
                _sum[c] += loglikelyhood[i, c] 
    return np.argmax(_sum)
        

In [65]:
V = list(inverted_word_index.values())

In [71]:
test_naive_bayes(decode_sentence(x_train[1]), logprior, loglikelyhood, [0, 1], V)

0
1


0

In [72]:
y_train[1]

0